In [6]:
from model.hierarchical_self_attention_model import HSA_model, HSA_VAE
import tensorflow as tf
import yaml

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
hparam_file = open('configs/hyperparameters.yaml', mode='r')
hyperparameters = yaml.load(hparam_file, Loader=yaml.FullLoader)

In [4]:
hyperparameters

{'HSA_model': {'modality_indices': {'opp': [0, 9, 18, 27, 36, 45, 61, 77],
   'mex': [0, 3, 6]},
  'dff': 512,
  'd_model': 64,
  'num_heads': 4,
  'dropout': 0.2},
 'train': {'batch_size': 64,
  'epochs': 10,
  'val_split': 0.1,
  'learning_rate': 0.001}}

In [5]:
n_window, n_timesteps, n_features, n_outputs = 36, 25, 77, 6

hsa_model = HSA_model(hyperparameters['HSA_model']['modality_indices']['opp'], n_window, n_timesteps, n_features, n_outputs, d_model=hyperparameters['HSA_model']['d_model'], num_heads = hyperparameters['HSA_model']['num_heads'], dff=hyperparameters['HSA_model']['dff'], dropout_rate=hyperparameters['HSA_model']['dropout'])

model = hsa_model.get_model()

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics='accuracy')
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 36, 25, 77)]      0         
_________________________________________________________________
multi_window_encoder (MultiW ((None, None, 64), (None, 1249472   
_________________________________________________________________
modality_encoder_block_7 (Mo (None, None, 64)          170048    
_________________________________________________________________
combined_sensor_self_attenti ((None, 64), (None, 1, No 82880     
_________________________________________________________________
dense_108 (Dense)            (None, 6)                 390       
Total params: 1,502,790
Trainable params: 1,502,790
Non-trainable params: 0
_________________________________________________________________


In [9]:
hsa_vae = HSA_VAE(base_model=model, feature_dim=64).get_model()

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
hsa_vae.compile(optimizer)    

In [6]:
metadata_file = open('configs/metadata.yaml', mode='r')
metadata = yaml.load(metadata_file, Loader=yaml.FullLoader)['opp_preprocess']

In [7]:
metadata['mid_label_to_id']

{'0': 0,
 '406516': 1,
 '406517': 2,
 '404516': 3,
 '404517': 4,
 '406520': 5,
 '404520': 6,
 '406505': 7,
 '404505': 8,
 '406519': 9,
 '404519': 10,
 '406511': 11,
 '404511': 12,
 '406508': 13,
 '404508': 14,
 '408512': 15,
 '407521': 16,
 '405506': 17}

In [33]:
!python dataset_download.py pamap2

2020-09-16 21:50:45
GETTING DATASET [pamap2.zip] ...
Unzipping [pamap2.zip] ...

---DATASET DOWNLOAD COMPLETE---


In [28]:
from preprocessing.utils import *

In [34]:
(X_train, y_train),  (X_test, y_test) = get_train_test_data('pamap2')

/home/agency/saif-tanjid/hierarchical-attention-HAR/preprocessing/pamap2_preprocess.py:19: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(output_file_name)


Done.
x_train shape =  (1692340, 18)
y_train shape = (1692340,)
x_val shape =  (0,)
y_val shape = (0,)
x_test shape = (250048, 18)
y_test shape = (250048,)
x_train shape =  (1692340, 18)
y_train shape = (1692340,)
x_val shape =  (0,)
y_val shape = (0,)
x_test shape = (250048, 18)
y_test shape = (250048,)


TypeError: create_windowed_dataset() missing 1 required positional argument: 'class_label'

In [ ]:
X_train.shape

In [12]:
verbose, epochs, batch_size = 1, 30, 32
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
              verbose=verbose)

Epoch 1/30


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model_1/multi_window_encoder/model/modality_encoder_block_5/data_transform_5/conv1d_5/conv1d (defined at /home/agency/saif-tanjid/final-code/hierarchical-attention-HAR-master/model/modaility_encoder.py:19) ]] [Op:__inference_train_function_40871]

Function call stack:
train_function
